<span style="color:blue">*magic_data*</span>

# Формирование портрета потенциального клиента для Ростелекома

от команды magic_data

**Цель исследования:** на основе выявленных закономерностей в данных о продажах потенциальным клиентам за прошлые периоды сформировать список рекомендаций для поиска новых клиентов. 

**Задачи исследования:**
1. Провести предобработку данных и EDA, включая:
- предварительный анализ данных на предмет наличия пропусков, дубликатов, ошибок;
- анализ показателей описательной статистики по имеющимся признакам;
- выявление аномалий и выбросов;
- предварительный анализ корреляции между признаками, устранение мультиколлинеарности;
- преобразование данных (стандартизация, перевод категориальных переменных в бинарный вид);
- формулировка гипотез, выбор моделей и алгоритмов машинного обучения;

2. Проанализировать временной ряд на наличие автокорреляции и тренда.


3. Реализовать алгоритмы машинного обучения, в том числе:
- провести разбиение выборки две части: train/test;
- обучить модели;
- сделать прогнозы;
- оценить метрики моделей;
- выбрать лучшие модели и на их основе сформировать разметку новых клиентов по склонности к подключению.

4. Составить портрет потенциального клиента с помощью описательной статистики.


5. Визуализировать результаты анализа.


6. Сформировать рекомендации по будущим продажам по каждому региону.

**Описание данных:**

Датасет содержит данные о продажах потенциальным клиентам за прошлые периоды.

**Показатели:**

- 
-
-
-
-

## 1. Предобработка данных и EDA

### Импортируем библиотеки и загрузим файлы

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats as st
import math as mth
import plotly.express as px
import datetime as dt

#from pandas.plotting import register_matplotlib_converters
#from urllib.parse import urlencode
#import requests
#import json
#from folium import Map, Choropleth, Marker
#from folium.plugins import MarkerCluster

from statsmodels.tsa.seasonal import seasonal_decompose, adfuller
from pylab import rcParams
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARMA, ARIMA
import statsmodels.api as sm

#import warnings
#from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LinearRegression, Ridge, LogisticRegression
from sklearn.preprocessing import StandardScalerfrom
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage 
from sklearn.decomposition import PCA

In [ ]:
data = pd.read_csv('', sep='')
    
data.info()

In [ ]:
old_data_shape = data.shape[0]
print('Исходное количество единиц наблюдения:', old_data_shape)

In [ ]:
pd.set_option('display.max_columns', 50)

In [ ]:
data.head(10)

### Анализ пропусков

In [ ]:
print('Количество пропусков по столбцам:')
data.isna().sum()

### Анализ дубликатов

In [ ]:
print('Количество явных дубликатов:', data.duplicated().sum())

In [ ]:
#анализ неявных дубликатов и ошибок в категориальных переменных
data['!!!'].unique()

### Анализ выбросов и описательной статистики

In [ ]:
data.hist(figsize=(15, 20));

**По гистограммам видно, что артефакты и выбросы могут быть в следующих показателях:**

- 
-
-
-
-

In [ ]:
plt.figure(figsize=(15, 4))
data.boxplot('')
plt.show()

In [ ]:
#выбросы можно поотбрасывать с помощью перцентилей
np.percentile(data[''], [90, 95, 99])

In [ ]:
#анализ описательной статистики
data.describe()

In [ ]:
print(
      'Коэффициент вариации:{:.2%}'.format(
       np.std(data[''])/data[''].mean()
        )
      )

In [ ]:
plt.figure(figsize=(20, 5))
sns.boxplot(x='', y='', data=)

plt.show()

In [ ]:
#приблизим
plt.figure(figsize=(20, 5))
sns.boxplot(x='platform', y='total_sales', data=relevant_data_var)
plt.ylim(0, 3)

plt.show()

### Анализ мультиколлинеарности

In [ ]:
#матрица корреляции
corr_matrix = data.corr()
fig, ax = plt.subplots()

In [ ]:
#тепловая карта для матрицы корреляции
sns.heatmap(corr_matrix, annot=True, square=True)
ax.set_ylim(7, 0)
plt.show()

In [ ]:
#диаграммы рассеяния для взаимосвязи показателей с целевой переменной
for col in data.drop('Целевая переменная', axis = 1).columns:
    sns.scatterplot(data=data, x=col, y=cars['Целевая пересенная'])
    plt.show()

In [ ]:
#преобразование категориальных переменных в бинарный вид

data[] = pd.get_dummies(data[])
data.head()

### Выводы по разделу:

1. 
2. 

## 2. Анализ временного ряда

Для начала посмотрим на динамику показателя ??? и его скользящее среднее за ???

In [ ]:
plt.figure(figsize = (15,8))
plt.plot(data[''], label = '???', color = 'steelblue')
plt.plot(data[''].rolling(window = 12).mean(), label = 'Скользящее среднее за 12 месяцев', color = 'orange')
plt.legend(title = '', loc = 'upper left', fontsize = 14)
plt.xlabel('Месяцы???', fontsize = 14)
plt.ylabel('???', fontsize = 14)
plt.title('???', fontsize = 16)
plt.show()

**Разложение ряда на компоненты**

In [ ]:
rcParams['figure.figsize'] = 11, 9
decompose = seasonal_decompose(data[''])
decompose.plot()
plt.show()

**Проверка на стационарность**

In [ ]:
# тест Дики-Фуллера
adf_test = adfuller(data[''])
print('p-value = ' + str(adf_test[1]))

Стационарен, если есть p-value меньше 0,05 - можем отвергнуть нулевую гипотезу о нестационарности процесса.

Если стационарен - тренд можно не выявлять.

**Проверка наличия автокорреляции**

In [ ]:
plot_acf(data[''], lags=np.arange(12))
plt.show()

In [ ]:
#уточним значение для лага, с наибольшей автокорреляцией
np.round(np.corrcoef(data[''][:-1], data[''][1:])[0,1], 2)

**Выделение тренда**

*Экспоненциальное сглаживание*

In [ ]:
alpha = 0.2
 
exp_smoothing = [data[''][0]]
for i in range(1, len(data[''])):
    exp_smoothing.append(alpha * data[''][i] + (1 - alpha) * exp_smoothing[i - 1])

In [ ]:
#создание датафрейма с экспоненциальным сглаживанием
trend_exp_sm = data[['Дата', '&&&']]
trend_exp_sm['Exp_smoothing'] = exp_smoothing
last_date = trend_exp_sm['Дата'].iloc[[-1]]
last_date = last_date + timedelta(days = 1)
trend_exp_sm = trend_exp_sm.append(pd.DataFrame(trend_exp_sm['Дата'] = last_date))
trend_exp_sm['Exp_smoothing'] = trend_exp_sm['Exp_smoothing'].shift(1)
trend_exp_sm.head()

In [ ]:
trend_exp_sm.tail()

In [ ]:
#график
plt.figure(figsize = (15,8))
plt.plot(trend_exp_sm[''], label = '', color = 'steelblue')
plt.plot(trend_exp_sm['Exp_smoothing'], label = 'Экспоненциальное сглаживание', color = 'orange')
plt.legend(title = '', loc = 'upper left', fontsize = 14)
plt.ylabel('', fontsize = 14)
plt.xlabel('Месяцы???', fontsize = 14)
plt.title('???', fontsize = 16)
 
plt.show()

*ARMA (для стационарных данных)*

In [ ]:
#датасет для анализа динамики
data_for_models = data[['Дата', '']]
data_for_models.set_index('Дата', inplace=True)

In [ ]:
#модель ARMA
model_arma = ARMA(data_for_models[''],order=(2,2))
model_arma_fit = model_arma.fit()

model_arma_fit.summary()

In [ ]:
#сравнение с реальным значением
data_for_models['forecast_ARMA'] = model_arma_fit.predict(start = !!!92, end= !!!114, dynamic= True)  
data_for_models[['', 'forecast_ARMA']].plot(figsize=(8, 5))

*ARIMA (для нестационарных данных)*

In [ ]:
model_arima = ARIMA(data[''],order=(2,1,2))
model_arima_fit = model_arima.fit()
model_arima_fit.summary()

In [ ]:
data_for_models['forecast_ARIMA'] = model_arima_fit.predict(start = 92, end= 114, dynamic= True)  
data[['', 'forecast_ARIMA']].plot(figsize=(8, 5))

*SARIMA (учитывает сезонность)*

In [ ]:
#!!!Здесь проверить точно ли это SARIMA, а не SARIMAX
model_sarima = sm.tsa.statespace.SARIMAX(data[''],order=(2,1,2),seasonal_order=(2,1,2,6))
model_sarima_fit = model_sarima.fit()
model_sarima_fit.summary()

In [ ]:
data['forcast_SARIMA'] = model_arima_fit.predict(start=99, end=112, dynamic=True)
data[['','forcast_SARIMA']].plot(figsize=(20, 5))

*SARIMAX (учитывает экзогенные факторы)*

In [ ]:
train = data_for_models[:'1959-12']
test = data_for_models['1960-01':]

#warnings.simplefilter(action = 'ignore', category = Warning)
 
model_sarimax = SARIMAX(train, order = (3, 0, 0), seasonal_order = (0, 1, 0, 12))
model_sarimax_fit = model.fit()
model_sarimax_fit.summary()

In [ ]:
start = len(train)
end = len(train) + len(test) - 1
predictions = model_sarimax_fit.predict(start, end)

plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.plot(predictions, color = "green")
plt.title("Обучающая выборка, тестовая выборка и тестовый прогноз")
plt.ylabel('&&&')
plt.xlabel('Месяцы')
plt.grid()
plt.show()

Для лучшией модели посмотри еще метрики среднеквадратической ошибки (MSE) и корня среднеквадратической ошибки (RMSE) для оценки качества.

In [ ]:
print('Среднеквадратическая ошибка (MSE): ', mean_squared_error(test, predictions))
print('Корень среднеквадратической ошибки (RMSE): ', np.sqrt(mean_squared_error(test, predictions)))

**Прогнозирование**

In [ ]:
#Создание последующих дат для прогнозирования

import datetime
from dateutil.relativedelta import relativedelta
start = datetime.datetime.strptime("1969-07-01", "%Y-%m-%d")
date_list = [start + relativedelta(months=x) for x in range(0,12)]
future_prediction = pd.DataFrame(index=date_list, columns= data.columns)
data = pd.concat([data, future])

In [ ]:
#прогнозирование по лучшей модели

data['future_prediction']=result.predict(start=113, end=130, dynamic=True)
data[['avg monthly busride','future_prediction']].plot(figsize=(10, 6))
plt.grid(True)

**Выводы по разделу:**

1. 
2. 

## 3. Реализация алгоритмов машинного обучения

Реализуем следующие алгоритмы машинного обучения:

- линейная регрессия (стандартная, а также с L1- и с L2-регуляризацией)
- логистическая регрессия (с решателями liblinear, sag и saga),
- дерево принятия решений,
- случайный лес,
- градиентный бустинг.

Cначала разделим выборки на обучающую/валидационную, проведем стандартизацию данных. Далее сделаем прогноз, сравним метрики и выберем лучшие модели.

### Обучение с учителем: регрессия

!!! Здесь попробуем добавить период в список факторов, чтобы невилировать влияние тренда, если таковой имеется.

In [ ]:
#разделение на x/y
X = data.drop('Целевая переменная', axis = 1)
y = data['Целевая переменная']


#Здесь скорее всего, будет привязка ко времени, нужно будет разделить с ее учетом
y_train = data[''][data['Дата'] < '']
y_test = data[''][data['Дата'] >= '']
X_train = data[data['Дата'] < ''].drop([''], axis = 0)
X_test = data[data['Дата'] >= ''].drop([''], axis = 0)

#или
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#стандартизация
scaler = StandardScaler()
scaler.fit(X_train)
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)

In [ ]:
#зададим функцию MAPE (средняя абсолютная процентная ошибкя):
def calc_mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
#зададим функцию для расчета разных моделей линейной регрессии:
def liner_model_func(m, X_train_st, X_test_st, y_train, y_test):
    model = m
    model.fit(X_train_st, y_train)
    pred = model.predict(X_test_st)
    features = pd.DataFrame({'feature': X_train.columns, 'coeff': model.coef_})
    features['coeff_abs'] = abs(features['coeff'])
    intercept = model.intercept_
    features = featureso.sort_values(by=['coeff_abs'], ascending=False)
    # вывод метрик
    print('\n', m, '\n')
    print('MAE (средний модуль ошибки): {:.2f}'.format(mean_absolute_error(y_test, pred)))
    print('MSE (средняя квадратичная ошибка): {:.2f}'.format(mean_squared_error(y_test, pred)))
    print('RMSE (корень из MSE): {:.2f}'.format(mean_squared_error(y_test, pred)**0.5))
    print('R2: {:.2f}'.format(r2_score(y_test, pred)))
    print('MAPE (средняя абсолютная процентная ошибкя): {:.2f}'.format(mape(y_test, pred)))

In [ ]:
#стандартная линейная регрессия
liner_model_func(LinearRegression(), X_train_st, X_test_st, y_train, y_test)

In [ ]:
#Lasso-регрессия (L1-регуляризация):
liner_model_func(Lasso(), X_train_st, X_test_st, y_train, y_test)

In [ ]:
#Ridge-регрессия (L2-регуляризация):
liner_model_func(Ridge(), X_train_st, X_test_st, y_train, y_test)

In [ ]:
#Дерево принятия решений:
liner_model_func(DecisionTreeRegressor(), X_train_st, X_test_st, y_train, y_test)

In [ ]:
#Случайный лес:
liner_model_func(RandomForestRegressor(), X_train_st, X_test_st, y_train, y_test)

In [ ]:
#Градиентный бустинг:
liner_model_func(GradientBoostingRegressor(), X_train_st, X_test_st, y_train, y_test)

Лучшими метриками обладают ??? модели. Для дальнейшей интерпретации будем использовать ???

Проанализируем значимость признаков в модели.

In [ ]:
final_model = GradientBoostingRegressor(random_state=0) #или другая модель
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

feat_importance = pd.DataFrame(data = {'feature': X.columns, 'importances': final_model.feature_importances_})
final_data.sort_values(by='importances')

**Наиболее важными признаками оказались:**

1. 
2. 

### Обучение с учителем: бинарная классификация

Добавим в датафрейм бинарную целевую переменную, содержащую факт покупки. И заново разделим выборку и проведем стандартизацию.

In [ ]:
data['Факт покупки'] = [1 for i in data['???'] if i not 0]

print('Получившееся распределение по группам:\n', data['Факт покупки'].value_counts())

In [ ]:
#разделение на x/y
X_b = data.drop(['Факт покупки', 'Целевой пок-ль из линейной регр'], axis = 1)
y_b = data['Факт покупки']

#Здесь скорее всего, будет привязка ко времени, нужно будет разделить с ее учетом
X_b_train, X_b_test, y_b_train, y_b_test = train_test_split(X_b, y_b, test_size=0.2)

#стандартизация
scaler = StandardScaler()
scaler.fit(X_b_train)
X_b_train_st = scaler.transform(X_b_train)
X_b_test_st = scaler.transform(X_b_test)

In [ ]:
#зададим функцию для расчета разных моделей бинарной классификации:
def binary_class_func(m, X_b_train, X_b_test, y_b_train, y_b_test):
    model = m
    model.fit(X_b_train_st, y_b_train)
    probabilities = model.predict_proba(X_b_test)[:, 1]
    predictions = model.predict(X_b_test)
    print('\n', m, '\n')
    print('Accuracy (доля правильных ответов): {:.2f}'.format(accuracy_score(y_b_test, predictions)))
    print('Precision (точность): {:.2f}'.format(precision_score(y_b_test, predictions)))
    print('Recall (полнота!): {:.2f}'.format(recall_score(y_b_test, predictions)))
    print('F1: {:.2f}'.format(f1_score(y_b_test, predictions)))
    print('AUC-ROC (площадь под кривой ошибок): {:.2f}'.format(roc_auc = roc_auc_score(y_b_test, probabilities[:,1])))

In [ ]:
#стандартная логистическая регрессия
binary_class_func(LogisticRegression(solver='liblinear'), X_b_train_st, X_b_test_st, y_b_train, y_b_test)

In [ ]:
#логистическая регрессия с решателем sag (L2)
binary_class_func(LogisticRegression(solver='sag'), X_b_train_st, X_b_test_st, y_b_train, y_b_test)

In [ ]:
#логистическая регрессия с решателем saga
binary_class_func(LogisticRegression(solver='saga'), X_b_train_st, X_b_test_st, y_b_train, y_b_test)

In [ ]:
#дерево принятия решений
binary_class_func(DecisionTreeClassifier(), X_b_train_st, X_b_test_st, y_b_train, y_b_test)

In [ ]:
#случайный лес
binary_class_func(RandomForestClassifier(n_estimators = 100), X_b_train_st, X_b_test_st, y_b_train, y_b_test)

In [ ]:
#градиентный бустинг
binary_class_func(GradientBoostingClassifier(n_estimators = 100), X_b_train_st, X_b_test_st, y_b_train, y_b_test)

Лучшими метриками обладают ??? модели. Поэтому прогнозировать принадлежность потенциального клиента к тому или иному классу будем на основе ???


Также можно доработать модель логистической регрессии, поэкспериментировав с порогом отнесения к классу вероятных клиентов. Поскольку нам важно не потерять потенциального клиента, то порог будем постепенно уменьшать. В качестве пороговых значений будем использовать 0.45, 0.40, 0.35 и 0.30.

In [ ]:
model_log = LogisticRegression(solver='liblinear', random_state=0)
model_log.fit(X_b_train_st, y_b_train)
predictions = model_log.predict(X_b_test_st)
probabilities = model_log.predict_proba(X_b_test_st)[:,1]
print('Accuracy (доля правильных ответов): {:.2f}'.format(accuracy_score(y_b_test, predictions)))
print('Precision (точность): {:.2f}'.format(precision_score(y_b_test, predictions)))
print('Recall (полнота!): {:.2f}'.format(recall_score(y_b_test, predictions)))
print('F1: {:.2f}'.format(f1_score(y_b_test, predictions)))
print('AUC-ROC (площадь под кривой ошибок): {:.2f}'.format(roc_auc = roc_auc_score(y_b_test, probabilities[:,1])))

In [ ]:
thresholds = [0.45, 0.4, 0.35, 0.3]

for threshold in thresholds:
    custom_predictions = [0 if i < threshold else 1 for i in probabilities]
    print('Метрики для прогноза с порогом ', i)
    print('Accuracy: {:.2f}'.format(accuracy_score(y_b_test, custom_predictions)))
    print('Precision: {:.2f}'.format(precision_score(y_b_test, custom_predictions)))
    print('Recall: {:.2f}'.format(recall_score(y_b_test, custom_predictions)))
    print('F1: {:.2f}'.format(f1_score(y_b_test, custom_predictions)))

Выбор лучшей модели

### Обучение без учителя: кластеризация

Для кластеризации потенциальных клиентов реализуем алгоритм на основе метода к-средних (k-means).

In [ ]:
#стандартизация
sc = StandardScaler()
X_sc = sc.fit_transform(X)

#модель
km = KMeans(n_clusters=5, random_state=0)
labels = km.fit_predict(X_sc)
data['clusters'] = labels

print('Средние значения признаков по кластерам \n', data.groupby(['clusters']).mean())

In [ ]:
#график кластеризации
linked = linkage(X_sc, method = 'ward') 
plt.figure(figsize=(15, 10))  
dendrogram(linked, orientation='top')
plt.title('Иерархическая кластеризация клиентов')
plt.show() 

In [ ]:
# Метрика силуэта
print('Silhouette score (метрика силуэта): {:.2f}'.format(silhouette_score(X_sc, labels)))

Вывод

### Уменьшение размерности данных с помощью метода главных компонент

Для того чтобы лучше понять, как взаимосвязаны между собой факторы, влияющие на склонность клиетов к покупке воспользуемся методом PCA.

In [ ]:
# Применение PCA с 2мя компонентами
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_sc)

# Визуализация
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
plt.title("Визуализация данных с PCA")
plt.xlabel("Главная компонента 1")
plt.ylabel("Главная компонента 2")
plt.show()

Посмотрим, какие факторы больше всего повлияли на эти компоненты.

In [ ]:
# Получение собственных векторов (весов) главных компонент
eigen_vectors = pca.components_

# Визуализация весов для первых нескольких компонент


# !!!!! Здесь нужно будет доработать код. Если не получится, то вывести просто веса главных компонент.
plt.figure(figsize=(10, 5))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(eigen_vectors[i].reshape(имя_изображения), cmap='viridis')
    plt.title(f"Главная компонента {i + 1}")
    plt.axis('off')
plt.show()

In [ ]:
 Попробуем использовать метод главных компонент для улучшения качества кластеризации.

In [ ]:
# Кластеризация после применения PCA
kmeans_pca = KMeans(n_clusters=5)
labels_pca = kmeans_pca.fit_predict(X_pca)

# Визуализация результатов
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.scatter(X_sc[:, 0], X_sc[:, 1], c=labels, cmap='viridis')
plt.title("Кластеризация без PCA")
plt.subplot(122)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels_pca, cmap='viridis')
plt.title("Кластеризация после PCA")
plt.show()

Вывод: 

**Выводы по разделу:**

1. 
2. 

## 4. Портрет потенциального клиента

In [ ]:
data.groupby('').mean()

data.pivot_table(index='', values='', aggfunc=['',''])

Вывод

## 5. Визуализация

### Дашборд

Ссылка на дашборд:


[Дашборд "!!!"](https://public.tableau.com/app/profile/anna7527/viz/_16738687084650/Dashboard1?publish=yes "!!!") 


Дашборд содержит:
- ;
- ;
- .

### Визуализация в Unity3d

Ссылка:


Даш содержит:

In [2]:
#Загрузка файла для дашборда:
#data_by_purchase.to_csv('data_by_purchase.csv', index = False)

## 6. Презентация

Ссылка на презентацию:


[Презентация "Формирование портрета потенциального клиента для Ростелекома от команды magic_data"](https://disk.yandex.ru/i/1CwfdbHipeEUSg "Формирование портрета потенциального клиента для Ростелекома от команды magic_data") 


## Общий вывод

**Что можно еще сделать?**

I. Можно еще дополнительно поразбиратьcя в агломеративной кластеризации и DBSCAN, KNN, CatBoost и LightGBM.

II. Можно посчитать какой-нибудь агрегатный показатель на основе факторов, полученных методом главных компонент. Например, с помощью многомерной средней с весами в виде общностей. По каждому субъету РФ. И визуализировать с помощью фоновой картограммы Choropleth

III. Можно взять какие-то данные из внешних источников и дополнить понециал по регионам. Например, количество домохозяйств по субъектам РФ, не подключенных к РТК, или количество новых зданий в субъекте. Количество новых домов можно попробовать получить с сайта: https://dom.mingkh.ru/
Потом можно визуализировать с помощью маркеров из folium и с помощью той же Choropleth.

IV. Еще можно попробовать сделать анализ временных рядов (просто анализ динамики и попробовать спрогнозировать что-нибудь)

V. Можно рассчитать показатели продаж по полному кругу клиентов и по выделенным кластерам, проверить статистические гипотезы:

- средняя покупка у клиентов из разных кластеров разная (t-тест):
```
H_0: средняя покупка кластера 1 = средняя покупка кластера 2
H_1: средняя покупка кластера 1 ≠ средняя покупка кластера 2
alpha = 0.05
```
- доля клиентов, совершивших покупку, в разных кластерах разная (z-тест):
```
H_0: доля  клиентов, совершивших событие, в кластере 1 = доля клиентов, совершивших событие, в кластере 2
H_1: доля  клиентов, совершивших событие, в кластере 1 ≠ доля клиентов, совершивших событие, в кластере 2
alpha = 0.05
```

# Черновики

In [ ]:
#моя старая mape на случай, если новая не сработает
def mape_old(y_true, y_pred):
    y_error = y_true - y_pred
    y_error_abs = [abs(i) for i in y_error]
    perc_error_abs = y_error_abs / y_true
    mape = perc_error_abs.sum() / len(y_true)
    return mape

In [ ]:
#черновик - функция с return:
def liner_model_func(model, X_train_st, X_test_st, y_train, y_test):
    model = model
    model.fit(X_train_st, y_train)
    pred = model.predict(X_test_st)
    features = pd.DataFrame({'feature': X_train.columns, 'coeff': model.coef_})
    features['coeff_abs'] = abs(features['coeff'])
    intercept = model.intercept_
    features = featureso.sort_values(by=['coeff_abs'], ascending=False)
    # Метрики
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = mean_squared_error(y_test, pred)**0.5
    r2 = r2_score(y_test, pred)
    mape = calc_mape(y_test, pred)
    return pred, features, intercept, mae, mse, rmse, r2, mape

In [ ]:
#На случай, если будет лучшей модель бинарной классификации - дерево принятия решений. Что маловероятно.

tree_model = DecisionTreeClassifier(min_samples_leaf=500)
tree_model.fit(X_b_train_st, y_b_train)
y_pred = tree_model.predict(X_b_test_st) 

plt.figure(figsize = (20,15)) # задайте размер фигуры, чтобы получить крупное изображение
plot_tree(tree_model, filled=True, feature_names = X_b_train_st.columns, class_names = ['потенциальный клиент', 'не клиент'])
plt.show() 

In [ ]:
#попарные графики признаков с разметкой по кластерам

import itertools

# определим функцию отрисовки графиков попарных признаков для кластеров
def show_clusters_on_plot(df, x_name, y_name, cluster_name):
    plt.figure(figsize=(5, 5))
    sns.scatterplot(
        df[x_name], df[y_name], hue=df[cluster_name], palette='Paired'
    )
    plt.title('{} vs {}'.format(x_name, y_name))
    plt.show()

# задаём модель k_means с числом кластеров 3 и фиксируем значение random_state
km = KMeans(n_clusters=3, random_state=0)
# прогнозируем кластеры для наблюдений (алгоритм присваивает им номера от 0 до 2)
labels = km.fit_predict(x_sc)

# сохраняем метки кластера в поле нашего датасета
travel['cluster_km'] = labels 

# выводим статистику по средним значениям наших признаков по кластеру
print(travel.groupby(['cluster_km']).mean())

# отрисуем графики для попарных сочетаний признаков с разметкой по кластерам
col_pairs = list(itertools.combinations(travel.drop('cluster_km', axis=1).columns, 2))
for pair in col_pairs:
    show_clusters_on_plot(travel, pair[0], pair[1], 'cluster_km')

In [ ]:
#Среднее за год
data[''].resample(rule = 'AS').mean().head()

In [ ]:
#скользящее среднее
passengers.rolling(window = 3).mean().head()